In [1]:
# File for machine learning model - ElasticNet Regression - Finetuned


import pandas as pd
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

dataname = "elasticnet_regression_finetune"

DATA_PATH = "../../Datasets/Vehice dataset/Downsampled/Test/test_sampled.csv"
OUTPUT_PATH = f"../../Datasets/Evaluations/Regression/{dataname}.csv"

TEST_PATH = "../../Datasets/Vehice dataset/Downsampled/Test/test_sampled.csv"
TRAIN_PATH = "../../Datasets/Vehice dataset/Downsampled/Train/train.csv"
VALID_PATH = "../../Datasets/Vehice dataset/Downsampled/Valid/valid.csv"




In [2]:
df_samples = pd.read_csv(DATA_PATH)
df_test = pd.read_csv(DATA_PATH)
df_train = pd.read_csv(TRAIN_PATH)

df_test.drop(columns=['prediction', 'prompt'], inplace=True)
df_samples.drop(columns=['prediction', 'prompt'], inplace=True)
print(df_samples.head())

                                    name  year  selling_price  km_driven  \
0     Ford Figo Aspire 1.5 TDCi Titanium  2017         670000      70000   
1  Mahindra Scorpio VLX 2WD AIRBAG BSIII  2012         525000     120000   
2                 Maruti Swift Dzire VDI  2014         438999      81000   
3              Ford Figo Diesel Titanium  2010         144000      50000   
4                 Hyundai i10 Magna 1.1L  2008         185000     110000   

     fuel seller_type transmission                 owner     mileage   engine  \
0  Diesel  Individual       Manual           First Owner  25.83 kmpl  1498 CC   
1  Diesel  Individual       Manual           First Owner  12.05 kmpl  2179 CC   
2  Diesel      Dealer       Manual           First Owner   23.4 kmpl  1248 CC   
3  Diesel  Individual       Manual          Second Owner   20.0 kmpl  1399 CC   
4  Petrol  Individual       Manual  Fourth & Above Owner  19.81 kmpl  1086 CC   

   max_power               torque  seats  
0     99 bhp 

In [3]:

df_samples = df_samples.dropna(how='any')

df_samples['prediction'] = None
df_samples["prompt"] = f'{dataname}'
df_samples["prompt_name"] = f'{dataname}'
df_samples["model_name"] = f'{dataname}'


In [4]:
def pre_process(df):
    df['year'] = 2020 - df['year']
    df['mileage'] = df['mileage'].str.replace(' kmpl', '', regex=False)
    df['mileage'] = df['mileage'].str.replace(' km/kg', '', regex=False)
    df['mileage'] = pd.to_numeric(df['mileage'])

    df['engine'] = df['engine'].str.replace(' CC', '', regex=False)
    df['engine'] = pd.to_numeric(df['engine'])


    df['max_power'] = df['max_power'].str.replace(' bhp', '', regex=False)
    df['max_power'] = pd.to_numeric(df['max_power'])
    
    df = df.dropna(how='any')
    return df

df_train = pre_process(df_train)
df_test = pre_process(df_test)

print(df_samples.head())

                                    name  year  selling_price  km_driven  \
0     Ford Figo Aspire 1.5 TDCi Titanium  2017         670000      70000   
1  Mahindra Scorpio VLX 2WD AIRBAG BSIII  2012         525000     120000   
2                 Maruti Swift Dzire VDI  2014         438999      81000   
3              Ford Figo Diesel Titanium  2010         144000      50000   
4                 Hyundai i10 Magna 1.1L  2008         185000     110000   

     fuel seller_type transmission                 owner     mileage   engine  \
0  Diesel  Individual       Manual           First Owner  25.83 kmpl  1498 CC   
1  Diesel  Individual       Manual           First Owner  12.05 kmpl  2179 CC   
2  Diesel      Dealer       Manual           First Owner   23.4 kmpl  1248 CC   
3  Diesel  Individual       Manual          Second Owner   20.0 kmpl  1399 CC   
4  Petrol  Individual       Manual  Fourth & Above Owner  19.81 kmpl  1086 CC   

   max_power               torque  seats prediction  \
0

In [5]:
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

X_train = df_train.drop('selling_price', axis=1)
y_train = df_train['selling_price']
X_test = df_test.drop('selling_price', axis=1)
y_test = df_test['selling_price']

categorical_features = ['name', 'fuel', 'seller_type', 'transmission', 'owner', 'torque']
numerical_features = X_train.columns.difference(categorical_features)  

preprocessor = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(handle_unknown='ignore'), categorical_features)],
    remainder='passthrough'  
)

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', ElasticNet(random_state=42))
])

param_grid = {
    'model__l1_ratio': [0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1]
}
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_squared_error', verbose=1)
grid_search.fit(X_train, y_train)
best_l1_ratio = grid_search.best_params_['model__l1_ratio']

# Predictions
y_train_pred = grid_search.predict(X_train)
y_test_pred = grid_search.predict(X_test)

train_rmse = mean_squared_error(y_train, y_train_pred, squared=False)
test_rmse = mean_squared_error(y_test, y_test_pred, squared=False)
train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)

print(f"Training RMSE: {train_rmse}")
print(f"Test RMSE: {test_rmse}")
print(f"Training R^2: {train_r2}")
print(f"Test R^2: {test_r2}")



Fitting 5 folds for each of 7 candidates, totalling 35 fits


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:639: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 10491086813592.111, tolerance: 281682416844.1201
  model = cd_fast.sparse_enet_coordinate_descent(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:639: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 12221322376127.69, tolerance: 273966586251.75836
  model = cd_fast.sparse_enet_coordinate_descent(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:639: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 12908105295079.63, tolerance: 273196240087.5501
  model = cd_fast.sparse_

Training RMSE: 75815.86846722524
Test RMSE: 229249.8101528772
Training R^2: 0.9913595941959932
Test R^2: 0.8996625978064465


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:639: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 15886027710994.666, tolerance: 336683959268.5045
  model = cd_fast.sparse_enet_coordinate_descent(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [6]:
print(f"Best l1_ratio: {best_l1_ratio}")

Best l1_ratio: 1


In [7]:
df_samples['prediction'] = y_test_pred

In [8]:
print(df_samples.head())

                                    name  year  selling_price  km_driven  \
0     Ford Figo Aspire 1.5 TDCi Titanium  2017         670000      70000   
1  Mahindra Scorpio VLX 2WD AIRBAG BSIII  2012         525000     120000   
2                 Maruti Swift Dzire VDI  2014         438999      81000   
3              Ford Figo Diesel Titanium  2010         144000      50000   
4                 Hyundai i10 Magna 1.1L  2008         185000     110000   

     fuel seller_type transmission                 owner     mileage   engine  \
0  Diesel  Individual       Manual           First Owner  25.83 kmpl  1498 CC   
1  Diesel  Individual       Manual           First Owner  12.05 kmpl  2179 CC   
2  Diesel      Dealer       Manual           First Owner   23.4 kmpl  1248 CC   
3  Diesel  Individual       Manual          Second Owner   20.0 kmpl  1399 CC   
4  Petrol  Individual       Manual  Fourth & Above Owner  19.81 kmpl  1086 CC   

   max_power               torque  seats     prediction 

In [20]:
df_samples.to_csv(OUTPUT_PATH, index=False)


In [26]:

importances = model.feature_importances_

feature_names = preprocessor.get_feature_names_out()

import pandas as pd

feature_importances_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importances
})

feature_importances_df = feature_importances_df.sort_values(by='Importance', ascending=False)

print(feature_importances_df)


                                                Feature  Importance
2044                               remainder__max_power    0.738631
2040                                    remainder__year    0.184073
2041                               remainder__km_driven    0.032031
2043                                  remainder__engine    0.008113
2042                                 remainder__mileage    0.007149
...                                                 ...         ...
617   onehot__name_Jaguar XF 3.0 Litre S Premium Luxury    0.000000
618               onehot__name_Jeep Compass 1.4 Limited    0.000000
624        onehot__name_Jeep Compass 2.0 Longitude BSIV    0.000000
626        onehot__name_Jeep Wrangler 2016-2019 3.6 4X4    0.000000
627                       onehot__name_Kia Seltos HTE D    0.000000

[2046 rows x 2 columns]
